In [ ]:
#10 - 1 대통령 연설문 텍스트 마이닝
# 가장 많이 사용된 단어 알아보기
#1. 연설문 불러오기
moon = open('speech_moon.txt', encoding = 'UTF-8').read()
moon

In [ ]:
#2. 불필요한 문자 제거하기
# 문자 처리 패키지인 re의 sub()을 이용해 한글이 아닌 모든 문자를 공백으로 바꾼다
import re
moon = re.sub('[^가-힣]', ' ', moon)
moon

#[^가-힣]: 한글이 아닌 모든 문자를 의미하는 정규 표현식

In [ ]:
#3. 명사 추출하기
#konlpy.tag.Hannanum()의 nouns()를 이용해 문장에서 명사 추출 가능하다.

#hannanum 만들기
import konlpy
hannanum = konlpy.tag.Hannanum()

#명사 추출하기 ex)
hannanum.nouns("대한민국의 영토는 한반도와 그 부속도서로 한다")

In [ ]:
#연설문에서 명사 추출하기
nouns = hannanum.nouns(moon)
nouns

In [ ]:
#data frame으로 변경
import pandas as pd
df_word = pd.DataFrame({'word' : nouns})
df_word

In [ ]:
#4. 단어 빈도표 만들기
# 한 글자로 된 단어는 의미가 없는 경우가 많아서 제거하기로 했다. pd.str.len()을 이용해 단어의 글자 수를 나타낸 변수를 추가한 다음 두 글자 이상인 단어만 추출하면 된다.

df_word['count'] = df_word['word'].str.len()
df_word
#count 변수 추가

In [ ]:
#두 글자 이상인 단어만 남기기
df_word = df_word.query('count>= 2')
df_word.sort_values('count')

In [ ]:
#단어의 사용 빈도를 구하고 빈도순으로 정렬

#단어 빈도 구하기
df_word = df_word.groupby('word', as_index = False).agg(n = ('word', 'count')).sort_values('n', ascending = False)
#단어별 분리 #빈도 구하기
df_word

In [ ]:
#5. 단어 빈도 막대 그래프 만들기
#단어 빈도 상위 20개 추출
top20 = df_word.head(20)
top20

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.rcParams['font.family'] = 'AppleGothic'
print(plt.rcParams['font.family'])

plt.rcParams.update({'font.family' : 'AppleGothic', #한글 폰트
                   'figure.dpi' : '120', #해상도
                   'figure.figsize' : [6.5, 6]}) # 가로 세로 크기

sns.barplot(data = top20, y = 'word', x = 'n')

In [ ]:
#워드 클라우드 만들기
#2. 폰트 설정 - 배민 도현체
font = 'Dohyeon-Regular.ttf'

In [ ]:
#3. 단어와 빈도를 담은 딕셔너리 만들기
#데이터 프레임을 딕셔너리화해준다.
dic_word = df_word.set_index('word').to_dict()['n']
dic_word

In [ ]:
#4. 워드 클라우드 만들기
from wordcloud import WordCloud
wc = WordCloud(random_state = 1234,       # 난수 고정  
              font_path = font,           # 폰트 설정
              width = 400,                # 가로 크기
              height = 400,               # 세로 크기 
              background_color = 'white') # 배경색

In [ ]:
#워드 클라우드 만들기
img_wordcloud = wc.generate_from_frequencies(dic_word)

#워드 클라우드 출력하기
plt.figure(figsize = (10,10)) #가로 세로 크기 설정
plt.axis('off') #테두리 선 없애기
plt.imshow(img_wordcloud)

In [ ]:
#5.  워드 클라우드 모양 바꾸기
import PIL
icon = PIL.Image.open('cloud.png')

In [ ]:
import numpy as np
img = PIL.Image.new('RGB', icon.size, (255, 255, 255))
img.paste(icon, icon)
img = np.array(img)

In [ ]:
#워드 클라우드 만들기
wc = WordCloud(random_state = 1234,       # 난수 고정  
              font_path = font,           # 폰트 설정
              width = 400,                # 가로 크기
              height = 400,               # 세로 크기 
              background_color = 'white', # 배경색
              mask = img)                 # mask 설정          

In [ ]:
img_wordcloud = wc.generate_from_frequencies(dic_word)

plt.figure(figsize = (10,10))    #가로 세로 크기 설정
plt.axis('off')                  #테두리 선 없애기
plt.imshow(img_wordcloud)        #워드 클라우드 출력

In [ ]:
#6. 워드 클라우드 색깔 바꾸기 

wc = WordCloud(random_state = 1234,       # 난수 고정  
              font_path = font,           # 폰트 설정
              width = 400,                # 가로 크기
              height = 400,               # 세로 크기 
              background_color = 'white', # 배경색
              mask = img,                 # mask 설정       
              colormap = 'inferno')       # 컬러맵 설정                     

In [ ]:
img_wordcloud = wc.generate_from_frequencies(dic_word)

#워드 클라우드 출력하기
plt.figure(figsize = (10,10))    #가로 세로 크기 설정
plt.axis('off')                  #테두리 선 없애기
plt.imshow(img_wordcloud)        #워드 클라우드 출력

In [ ]:
#10 - 2 기사 댓글 텍스트 마이닝

#1. 가장 많이 사용된 단어 알아보기 

#데이터 불러오기
df = pd.read_csv('news_comment_BTS.csv', encoding = 'UTF-8')

df.info()

In [ ]:
#불필요한 문자 제거하기
#reply = 댓글
df['reply'] = df['reply'].str.replace('[^가-힣]', ' ',regex = True)
df['reply'].head()

In [ ]:
#명사 추출하기
#꼬꼬마 형태소 분석기
kkma = konlpy.tag.Kkma()

In [ ]:
#명사 추출- apply()활용
nouns = df['reply'].apply(kkma.nouns)
nouns

In [ ]:
#단어 빈도표 만들기
# 한 행에 한 단어가 들어가도록 구성
nouns = nouns.explode()
nouns

In [ ]:
#데이터 프레임 만들기
df_word = pd.DataFrame({'word':nouns})

#글자 수 추가
df_word['count'] = df_word['word'].str.len()

#두 글자 이상 단어만 남기기
df_word = df_word.query('count >= 2')
df_word

In [ ]:
#빈도표 만들기
#단어 빈도 구하기
df_word = df_word.groupby('word', as_index = False).agg(n = ('word', 'count')).sort_values('n', ascending = False)
#단어별 분리 #빈도 구하기
df_word

In [ ]:
#단어 빈도 상위 20개 추출
top20 = df_word.head(20)
top20

In [ ]:
#가로 세로 크기 설정
plt.rcParams.update({'figure.figsize' : [6.5, 6]})

#막대 그래프 만들기
sns.barplot(data = top20, y = 'word', x = 'n')

In [ ]:
#2. 워드 클라우드 만들기

dic_word = df_word.set_index('word').to_dict()['n']

In [ ]:
wc = WordCloud(random_state = 1234,
              font_path = font,
              width = 400,
              height = 400,
              background_color = 'white',
              mask = img)

In [ ]:
img_wordcloud = wc.generate_from_frequencies(dic_word)

plt.figure(figsize = (10,10))
plt.axis('off')
plt.imshow(img_wordcloud)

In [ ]:
#11 - 1 시군구별 인구 단계 구분도 만들기

#1. 시군구별 인구 단계 구분도 만들기
import json
geo = json.load(open('SIG.geojson', encoding = 'UTF-8'))

In [ ]:
#행정구역 코드 출력
geo['features'][0]['properties']

In [ ]:
#위도, 경도 좌표 출력
geo['features'][0]['geometry']

In [ ]:
# 시군구별 인구 데이터 준비하기
import pandas as pd

df_pop = pd.read_csv('Population_SIG.csv')
df_pop.head()

In [ ]:
df_pop.info()

In [ ]:
#행정구역 코드가 문자 타입으로 되어 있어야 지도 만드는데 활용할 수 있다.
df_pop['code'] = df_pop['code'].astype('str')

In [ ]:
#단계 구분도 만들기

import folium

folium.Map(location = [35.95, 127.7], # 지도 중심 좌표
          zoom_start = 8)             # 확대 단계

In [ ]:
map_sig = folium.Map(location = [35.95, 127.7],
                    zoom_start = 8,
                    tiles = 'cartodbpositron')
map_sig

In [ ]:
#단계 구분도 만들기
folium.Choropleth(geo_data = geo,
                  data = df_pop,
                  columns = ('code', 'pop'),
                  key_on = 'feature.properties.SIG_CD').add_to(map_sig)

map_sig

In [ ]:
# 계급구간 정하기
# 분위수를 이용해 지역을 적당히 나눠 계급구간 정한다.

bins = list(df_pop['pop'].quantile([0, 0.2, 0.4, 0.6, 0.8, 1]))
bins

In [ ]:
# 디자인 수정하기
# 배경 지도 만들기

map_sig = folium.Map(location = [35.95, 127.7],
                    zoom_start = 8,
                    tiles = 'cartodbpositron')

In [ ]:
#단계 구분도 만들기
folium.Choropleth(
geo_data = geo, #지도 데이터
data = df_pop,  #통계 데이터
columns = ('code', 'pop'),  #df_pop 행정 구역 코드, 인구
key_on = 'feature.properties.SIG_CD',  #geo 행정 구역 코드
fill_color = 'YlGnBu', #컬러맵
fill_opacity = 1,  # 투명도
line_opacity = 0.5, # 경계선 투명도
    
bins = bins).add_to(map_sig) #계급 구간 기준값 #배경 지도에 추가
  
map_sig

In [ ]:
#11 - 2 서울시 동별 외국인 인구 단계 구분도 만들기
#서울시 동 경계 지도 데이터 준비하기

import json

geo_seoul = json.load(open('EMD_Seoul.geojson', encoding = 'UTF-8'))

In [ ]:
#행정 구역 코드 출력
geo_seoul['features'][0]['properties']

In [ ]:
#위도, 경도 좌표 출력
geo_seoul['features'][0]['geometry']

In [ ]:
#서울시 동별 외국인 인구 데이터 준비
foreigner = pd.read_csv('Foreigner_EMD_Seoul.csv')
foreigner.head()
#동별 행정구역 코드, 동 이름, 외국인 인구

In [ ]:
foreigner.info()

#행정구역 코드를 나타낸 foreigner의 code는 int64 type으로 되어있다

In [ ]:
#행정구역 코드 문자로 바꿔야 함
foreigner['code'] = foreigner['code'].astype(str)

In [ ]:
#단계 구분도 만들기

#지역을 8단계로 나누도록 계급구간의 하한값, 상한값을 만든다

bins = list(foreigner['pop'].quantile([0, 0.2, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]))
bins

In [ ]:
#서울이 가운데에 오도록 배경지도를 만들고 단계 구분도를 추가한다.
#인구가 많을수록 진한 파란색으로 표현한다.
#외국인 인구가 결측치인 부분은 흰색으로 표현

#배경 지도 만들기
map_seoul = folium.Map(location = [37.56, 127],  #서울 좌표
                      zoom_start = 12,           #확대 단계
                      tiles = 'cartodbpositron') #지도 종류

#단계 구분도 만들기
folium.Choropleth( 
geo_data = geo_seoul,                    #지도 데이터   
data = foreigner,                        #통계 데이터  
columns = ('code','pop'),                #foreigner 행정구역 코드, 인구 
key_on = 'feature.properties.ADM_DR_CD', #geo_seoul 행정구역 코드
fill_color = 'Blues',                    #컬러맵
nan_fill_color = 'White',                #결측치 색깔
fill_opacity = 1,                        #투명도
line_opacity = 0.5,                      #경계선 투명도
bins = bins).add_to(map_seoul)           #계급 구간 기준값, 배경 지도에 추가
map_seoul

In [ ]:
#구 경계선 추가하기
geo_seoul_sig = json.load(open('SIG_Seoul.geojson', encoding = 'UTF-8'))

In [ ]:
#서울시 구 경계선을 이용해 단계 구분도를 만든 다음, 앞에 만든 지도에 추가한다. 
#색을 칠하지 않도록 하고 경계선을 두껍게 나타내도록 한다.

#서울 구 경계선 추가
folium.Choropleth(geo_data = geo_seoul_sig,         #지도 데이터
                 fill_opacity = 0,                  #투명도
                 line_weight = 4).add_to(map_seoul) #선 두께, 지도에 추가
map_seoul